<a href="https://colab.research.google.com/github/Benish-Lezdo/ai_learnings/blob/main/colpoli_document_retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ColPali Example



```
# This is formatted as code
```



In [ ]:
!pip install --upgrade byaldi
!pip install  poppler-utils
!pip install -q pdf2image git+https://github.com/huggingface/transformers.git qwen-vl-utils flash-attn




  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 19.3 MB/s eta 0:00:00
ERROR: Operation cancelled by user


In [1]:

!pip install -q pdf2image
!pip install flash-attn

!sudo apt-get install -y poppler-utils

from byaldi import RAGMultiModalModel
from huggingface_hub import notebook_login
from pdf2image import convert_from_path

images = convert_from_path("/content/pdf/injury_report.pdf")
images[0]

  Using cached flash_attn-2.6.3.tar.gz (2.6 MB)
  Preparing metadata (setup.py) ... done
ERROR: Operation cancelled by user
^C


KeyboardInterrupt: 

In [17]:
import os
from byaldi import RAGMultiModalModel
from pathlib import Path
import torch
from torch.quantization import quantize_dynamic

# !wget https://arxiv.org/pdf/1706.03762
# !mkdir docs
# !mv 1706.03762 docs/attention.pdf
# !cp -r docs/attention.pdf docs/attention_with_a_mustache.pdf

model = RAGMultiModalModel.from_pretrained("vidore/colpali")
# model = RAGMultiModalModel.from_pretrained("vidore/colpali-v1.2")




index_name = "image_index"

model.index(
    input_path=Path("/content/pdf"),
    index_name= index_name,
    store_collection_with_index=False,
    overwrite=True,
)

ModuleNotFoundError: No module named 'byaldi'

#BioBERT model

In [16]:
import torch
from transformers import AutoTokenizer, AutoModel
import torch.nn.functional as F

# Load the BioBERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.1")
model = AutoModel.from_pretrained("dmis-lab/biobert-base-cased-v1.1")


def embed_text(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)

    embeddings = outputs.last_hidden_state[:, 0, :]
    return embeddings

def split_into_sentences(document):
    sentences = document.split('. ')
    return sentences

def answer_query(document, query):
    chunks = split_into_sentences(document)
    query_embedding = embed_text(query)
    chunk_embeddings = [embed_text(chunk) for chunk in chunks]
    similarities = [F.cosine_similarity(chunk_embedding, query_embedding, dim=1) for chunk_embedding in chunk_embeddings]
    most_similar_chunk_idx = torch.argmax(torch.cat(similarities))

    return chunks[most_similar_chunk_idx]


document = """
Cardiovascular disease is the leading cause of death worldwide. It affects millions of people every year.
There are several types of cardiovascular diseases, including coronary artery disease and heart failure.
Prevention includes lifestyle changes, such as regular exercise and a healthy diet.
"""

query = "What are the prevention measures for cardiovascular disease?"

answer = answer_query(document, query)
print(f"Answer: {answer}")


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Answer: It affects millions of people every year.
There are several types of cardiovascular diseases, including coronary artery disease and heart failure.
Prevention includes lifestyle changes, such as regular exercise and a healthy diet.

